In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import re 
import joblib
import string  

In [5]:
fake = pd.read_csv('Fake.csv')
true = pd.read_csv('True.csv')

In [6]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [7]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [9]:
fake['class']=0
true['class']=1

In [10]:
data = pd.concat([fake,true], axis = 0)

In [11]:
data.sample(10)

,title,text,subject,date,class
13219,HOW MALIA OBAMA’S POT SMOKING FRIEND Is Connec...,Barack Obama is above the law his daughter is ...,politics,"Aug 18, 2016",0
12209,$3.5 MILLION DOWN THE DRAIN….Day 5 WI Recount:...,Just think how many youth centers could have b...,politics,"Dec 6, 2016",0
7526,Clinton leads Trump 47-43 percent in White Hou...,WASHINGTON (Reuters) - Democratic presidential...,politicsNews,"November 7, 2016",1
12343,NBA CRYBABY COACH Worries About President-Elec...,"It turns out the hypocrite, liberal, Golden S...",politics,"Nov 20, 2016",0
2261,"In One GLORIOUS Move, CNN REFUSES To Bow Down...","If there s one thing Donald Trump wants, it s ...",News,"March 6, 2017",0
18522,EU's Tusk appealed to Rajoy to avoid escalatio...,BRUSSELS (Reuters) - European Council Presiden...,worldnews,"October 2, 2017",1
6129,Racist TX High Schoolers Spray Paint ‘Whites ...,Several high school seniors in Texas won t be ...,News,"May 28, 2016",0
15064,Obama Celebrates Iran Deal As Senior Iranian O...,Ever feel like a headline you re reading is to...,politics,"Oct 19, 2015",0
10032,Senators ask Treasury to probe U.S. links to '...,WASHINGTON (Reuters) - Senators Elizabeth Warr...,politicsNews,"April 7, 2016",1
3036,Inspector General To Investigate Comey Over P...,"In a bit of irony, the Inspector General (who ...",News,"January 12, 2017",0


In [12]:
data = data.drop(["title","subject","date"], axis = 1)

In [13]:
data.reset_index(inplace=True)

In [14]:
data.drop(['index'],axis = 1,inplace=True)

In [15]:
data.sample(5)

,text,class
37116,WASHINGTON (Reuters) - U.S. Secretary of State...,1
6792,The tried and true Donald Trump supporter does...,0
41570,ZURICH (Reuters) - A 38-year-old refugee from ...,1
32196,"(Reuters) - U.S. TV networks, buoyed by strong...",1
17545,Is there a single person left on Robert Muelle...,0


In [20]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    return text

In [21]:
data["text"] = data["text"].apply(clean_text)

In [22]:
x=data["text"]
y=data["class"]

xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size=0.25,random_state=42)

In [23]:
vectorizer = TfidfVectorizer()
xv_train = vectorizer.fit_transform(xtrain)
xv_test = vectorizer.transform(xtest)

In [24]:
lr = LogisticRegression()
lr.fit(xv_train,ytrain)

LogisticRegression()

In [25]:
prediction = lr.predict(xv_test)
lr.score(xv_test,ytest)

0.985924276169265

In [26]:
print(classification_report(ytest,prediction))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5895
           1       0.98      0.99      0.99      5330

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [27]:
joblib.dump(vectorizer,"vectorizer.jb")
joblib.dump(lr,"lr_model.jb")

['lr_model.jb']